In [ ]:
# pip install directly into notebook
%pip install sklearn
%pip install pandas
%pip install imbalanced-learn
%pip install seaborn
%pip install shap

In [ ]:
# TODO: consider making list of dependencies for TA to install when running this notebook
import sklearn as sk
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, KFold, cross_validate
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

import shap

In [ ]:
# disable warning for chained assignment (not necessary but cleans up the project)
pd.options.mode.chained_assignment = None

In [ ]:
# taken from kaggle example
class ReplaceZeroTransformer():
    """Eliminates Zero values from tempo columns and replace it 
       with the median or mean of non-zero values as specified.
       defaut is set to 'median'.
    """

    def __init__(self, method='median'):
        self.method = method

    def transform(self, X):
        if self.method == 'median':
            X.loc[X['tempo']==0, 'tempo'] = X.loc[X['tempo']>0, 'tempo'].median()
        elif self.method == 'mean':
            X.loc[X['tempo']==0, 'tempo'] = X.loc[X['tempo']>0, 'tempo'].mean()
        else:
            raise Exception("Method can be 'median' or 'mean' only!")
        return X

In [ ]:
# Import track data
usecols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode','popularity', 'speechiness', 'tempo', 'valence']
dataset = pd.read_csv("data.csv", header = 0, usecols=usecols)

# Remove rows duplicated by ignoring some columns
dataset = dataset[~dataset.duplicated()==1]

# Normalize columns having values outside [0, 1]
scaler = MinMaxScaler()
# cols_to_normalize = ['duration_ms', 'key', 'loudness', 'popularity', 'tempo']
cols_to_normalize = ['duration_ms', 'loudness', 'tempo']
dataset[cols_to_normalize] = scaler.fit_transform(dataset[cols_to_normalize])

# print(dataset)

In [ ]:
# Data analysis
# pair plots: https://seaborn.pydata.org/generated/seaborn.pairplot.html
import seaborn as sb

# Pairwise plots between each column in the data set excluding popularity, binary (mode and explicit) and discrete (key) features
pair_plot_data = dataset.drop(columns=['mode', 'explicit', 'key', 'popularity'])
sb.pairplot(pair_plot_data, corner=True)
# There isn't a lot of direct correlation between any two property of a song

# Correlation between each property and popularity
sb.pairplot(dataset, x_vars=usecols, y_vars=['popularity'])
# The proprties we have do not directly correlate with the popularity of a song

# Analyze binary and discrete features (mode, explicit, key)
sb.displot(data=dataset, x='mode')        # more songs are in major key
sb.displot(data=dataset, x='explicit')    # most songs are non explicit, most likely do not need this feature
sb.displot(data=dataset, x='key')         # somewhat even distribution of keys

# Analyze features in relation to popularity; these graphs are not very useful
sb.pairplot(dataset, y_vars=['mode', 'explicit', 'key'], x_vars=['popularity'])

# Count non-continuous features
non_explicit = dataset[dataset['explicit'] == 0].shape[0]
major = dataset[dataset['mode'] == 1].shape[0]
total_songs = dataset.shape[0]
print(f"Non-Explicit songs / total songs = {non_explicit/total_songs}")
print(f"Major key songs / total songs = {major/total_songs}")

# Analyze range of popularity of songs given explicit = 0 or 1 and mode = 0 or 1
sb.histplot(data=dataset, x="popularity", hue="explicit", multiple="dodge")
# sb.histplot(data=dataset, x="popularity", hue="mode", multiple="dodge")    # similar shapes for both


In [ ]:
# TODO: further preprocessing?

y = dataset.pop('popularity') # popularity is our class to predict
X_headers = list(dataset.columns.values)
X = dataset

# Create the under sampler
undersample = RandomUnderSampler(sampling_strategy='majority')

# apply the transform
X, y = undersample.fit_resample(X, y)

tempo_transformer = ReplaceZeroTransformer()
X = tempo_transformer.transform(X)

# need to scale after to treat the individual categories as their own class for the undersampling
y = y/100

In [ ]:
# one hot encode the keys since they are a multiclass
ohe = OneHotEncoder(categories='auto', drop='first')

feature_arr = ohe.fit_transform(X[['key']]).toarray()
columns_key = ['key_'+str(i) for i in list(set(X['key'].values))[1:]]
features = pd.DataFrame(feature_arr, columns = columns_key, index = X.index)
X = pd.concat([X, features], axis=1).drop(['key'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
# Hyperparameter tuning

# Set the parameters by cross-validation
tuned_parameters = [
    # {
    #     'max_depth': np.arange(5, 15),
    # },
    # {
    #     'kernel': ["linear"],
    # },
    {
        'kernel': ["rbf"],
        'C': np.linspace(0.1, 1, 5)
    },
        # {
        # 'kernel': ["sigmoid"],
        # 'coef0': np.arange(0, 5),
        # 'C': np.linspace(0.1, 1, 5)
    # },
    {
        'kernel': ["poly"],
        'coef0': np.arange(0, 5),
        'degree': np.arange(3, 5),
        'C': np.linspace(0.1, 1, 5),
    },
]

# Available regression metrics are given here: https://scikit-learn.org/stable/modules/classes.html#regression-metric
# https://stackoverflow.com/questions/42228735/scikit-learn-gridsearchcv-with-multiple-repetitions/42230764#42230764
# ensure scikit is >0.18


inner_cv = KFold(n_splits=5, shuffle=True, random_state=0)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=1)
print("Tuning hyper-parameters begin!")
print()

# clf = GridSearchCV(tree.DecisionTreeRegressor(), tuned_parameters, cv=inner_cv, scoring='neg_mean_squared_error', verbose=4, n_jobs=3)
clf = HalvingGridSearchCV(svm.SVR(verbose=True,cache_size=3000), tuned_parameters, cv=inner_cv, scoring='neg_mean_squared_error', verbose=4, n_jobs=3, random_state=1)
print("Classifiers established, training data")
print()

clf.fit(X, y)
non_nested_scores = clf.best_score_
print("Best parameters found:", clf.best_params_)
print("Score (mean squared):", -clf.best_score_)

print("Running cross validation")
print()
clf.best_params_["cache_size"] = 3000
clf.best_params_["verbose"] = True

# cross_val_raw_data = cross_validate(clf, X=X, y=y, cv=outer_cv, verbose=4,  n_jobs=3, return_estimator=True, return_train_score=True)
clf = svm.SVR(**clf.best_params_)
cv_score = cross_val_score(clf, X=X, y=y, cv=outer_cv, verbose=4,  n_jobs=3,  scoring='neg_mean_squared_error')
print("Cross validation score (mean squared):", -cv_score.mean())

clf.fit(X, y)

# print('Feature importances:')
# for i, col in enumerate(X.columns):
#     print(f'{col:18}: {clf.feature_importances_[i]:.3f}')

In [ ]:
# Based on the KernelExplainer example available at https://github.com/slundberg/shap
shap.initjs()
hyperparameters = {'cache_size'=3000, 'C': 0.325, 'coef0': 3, 'degree': 3, 'kernel': 'poly'}
svm = svm.SVR(**hyperparameters)
svm.fit(X_train, y_train)

explainer = shap.KernelExplainer(svm.predict_proba, X_train, link="logit")
shap_values = explainer.shap_values(X_test, nsamples=100)

# plot the SHAP values for the Setosa output of the first instance
shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")